Opción B

In [25]:
import os
from together import Together

client = Together(api_key=OPEN_MODELS_KEY)

response = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[{"role": "user", "content": "Tell me fun things to do in New York"}],
)
print(response.choices[0].message.content)

The city that never sleeps! New York City is a treasure trove of excitement, entertainment, and adventure. Here are some fun things to do in the Big Apple:

**Iconic Landmarks & Attractions**

1. **Statue of Liberty and Ellis Island**: Take a ferry to Liberty Island to see the iconic statue up close and visit the Ellis Island Immigration Museum.
2. **Central Park**: A tranquil oasis in the middle of Manhattan, perfect for a stroll, picnic, or bike ride.
3. **Empire State Building**: For a panoramic view of the city, head to the observation deck of this iconic skyscraper.
4. **The Metropolitan Museum of Art**: One of the world's largest and most famous museums, with a collection that spans over 5,000 years of human history.
5. **Times Square**: The heart of New York's entertainment district, known for its bright lights, giant billboards, and lively street performers.

**Food & Drink**

1. **Try a classic New York-style pizza**: Visit Lombardi's, Joe's Pizza, or Patsy's Pizzeria for a sl

# Together AI + Langchain - embeddings -> database

In [4]:
from langchain_together.embeddings import TogetherEmbeddings

def get_embedding_function():
    
    os.environ["TOGETHER_API_KEY"] = 'b065ada4924b7209869006860cca59b71a6a3835a2cd9581c772ae571aa38c8f'
    embeddings = TogetherEmbeddings(model="WhereIsAI/UAE-Large-V1")
    
    return embeddings

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader

In [5]:
pdfpath="C:\\Users\\Enrique\\Desktop\\CHATBOT\\data\\RPCOUNTER_USER_MANUAL_es_ES.pdf"
loader = PyPDFLoader(pdfpath)
docs = loader.load()

In [2]:
def get_pdf_text(pdf_docs):
    text=""
    #for pdf in pdf_docs:
    pdf_reader= PdfReader(pdf_docs)
    for page in pdf_reader.pages:
        text+= page.extract_text()
    return  text

In [6]:
docs=get_pdf_text(pdfpath)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
data = text_splitter.split_text(docs)

In [6]:
from langchain.vectorstores import Chroma
CHROMA_PATH = "C:\\Users\\Enrique\\Desktop\\CHATBOT\\chroma"
import os
# Set the API key in your environment variables or directly in your script
os.environ['TOGETHER_API_KEY'] = 'b065ada4924b7209869006860cca59b71a6a3835a2cd9581c772ae571aa38c8f'

RAG pruebas resultados online

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
template = """You are a bot that answers questions about the context provided.
If you don't know the answer, simply state that you don't know.

{context}

Question: {question}"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [ ]:
qa_with_source = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT, },
    return_source_documents=True,
)

In [ ]:
qa_with_source.invoke("Qué es Inrobics?")

RAG pruebas con Ollama

In [ ]:
import argparse
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

from llm_utils import get_embedding_function

CHROMA_PATH = "C:\\Users\\Enrique\\Desktop\\CHATBOT\\chroma_ESNOVA3"

PROMPT_TEMPLATE_ENG = """

You are a bot that answers questions about the context provided.
If you don't know the answer, simply state that you don't know.

Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

PROMPT_TEMPLATE = """

Eres un bot que responde a preguntas sobre el contexto que te aporto, extraído de un manual de usuario. 
En concreto, respondes preguntas sobre un manual de usuario.
Si no sabes la respuesta con mucha seguridad, simplemente indica que no estás en capacidad de responderla, y sugiere que se pruebe a preguntar de nuevo.
PROHIBIDO empezar con frases para citar la referencia, da la respuesta y NADA MÁS. El usuario cree que habla con el mismo manual en PDF.

Responde basándote únicamente en:

{context}

---

Responde a esta pregunta basándote en lo anterior: {question}


"""


def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=25)

    print(results)
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    # AQUÍ CAMBIA SI USO LLAMA
    model = Ollama(model="llama3")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    return formatted_response

In [20]:
query_text = 'Cómo mover el pórtico'
query_text = 'Cómo realizar una actualización de software'
query_text = 'Qué significa el color rojo en los resultados del historial'
query_text = "Resúmeme el apartado 'rack paletizado'. responde en polaco"
query_text = "Cuáles son los tipos de pallet de unidad de carga. dame una lista con todos los puntos. responde en polaco, traduce bien lo que uses del contexto"

print(query_rag(query_text))

[(Document(page_content='4.7.  Lista de materiales  ................................ ................................ ................  41 \n4.8.  Memoria de oferta  ................................ ................................ ...............  43 \n4.9.  Plano  ................................ ................................ ................................ ...... 44 \n4.10.  Almacén  ................................ ................................ .............................  49', metadata={'id': 'Manual de usuario EsnovaRacks Español V2201.pdf:3:4', 'page': 3, 'source': 'C:\\Users\\Enrique\\Desktop\\CHATBOT\\data\\Manual de usuario EsnovaRacks Español V2201.pdf'}), 167.46009826660156), (Document(page_content='cargas sean las mismas.  \n \nSolamente los usuarios avanzados pueden asignar p erfiles de \npuntal distintos en una misma alineación. Si tiene una alineación \nsingular que pueda necesitar este tipo de solución, póngase en \ncontacto con nuestro Departamento Comercial 

Evaluar técnica con preguntas

Extras: asegurarasd de que solo siga un propósito claro, sino que se niegue. incluso modelo de safeguard